In [1]:
!pip install -U sentence-transformers


In [2]:
from sentence_transformers import SentenceTransformer


In [3]:
model = SentenceTransformer("all-MiniLM-L6-v2")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
text = """
Artificial intelligence (AI) refers to computational systems that can perform tasks
commonly associated with human intelligence, such as learning from data, recognizing
patterns, reasoning, and making decisions. In practice, AI is used to automate or
improve processes like text understanding, image recognition, and recommendation
systems.

Modern AI systems often rely on machine learning models trained on large datasets.
While these models can generalize patterns effectively, they do not inherently have
access to up-to-date or domain-specific information unless it is explicitly provided.
"""


In [5]:
embedding = model.encode(text)


In [6]:
type(embedding)
len(embedding)


384

In [7]:
text_2 = """
Artificial intelligence techniques are widely used in medical diagnosis
and healthcare analytics to support clinical decision-making.
"""

embedding_2 = model.encode(text_2)
len(embedding_2)


384

In [8]:
from numpy import dot
from numpy.linalg import norm

similarity = dot(embedding, embedding_2) / (norm(embedding) * norm(embedding_2))
similarity


np.float32(0.5576549)

In [9]:
!pip install -U faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 30.2 MB/s eta 0:00:00


In [10]:
import faiss
import numpy as np


In [11]:
import faiss
import numpy as np

# Convert the embedding to a format compatible with FAISS
embeddings = np.array([embedding]).astype("float32")

# Create a simple FAISS index using L2 (Euclidean) distance
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)

# Add the embedding vector to the index
index.add(embeddings)

# Number of vectors stored in the index
index.ntotal


1

In [12]:
texts = [
    "Artificial intelligence focuses on building intelligent systems.",
    "AI is widely used in medical diagnosis and healthcare.",
    "Football is a popular sport played worldwide."
]

embeddings_list = model.encode(texts)
embeddings_list = np.array(embeddings_list).astype("float32")

embeddings_list.shape


(3, 384)

In [13]:
# Query text
query = "Applications of AI in healthcare"

# Create embedding for the query
query_embedding = model.encode(query)
query_embedding = np.array([query_embedding]).astype("float32")

# Search for the nearest vectors (top-2)
distances, indices = index.search(query_embedding, k=2)

distances, indices


(array([[9.0163702e-01, 3.4028235e+38]], dtype=float32), array([[ 0, -1]]))

In [14]:
for idx in indices[0]:
    print(texts[idx])


Artificial intelligence focuses on building intelligent systems.
Football is a popular sport played worldwide.


In [15]:
# Recreate FAISS index
index = faiss.IndexFlatL2(dimension)

# Add all embeddings
index.add(embeddings_list)


In [16]:
# Create new index and add all vectors
index = faiss.IndexFlatL2(dimension)
index.add(embeddings_list)


In [17]:
distances, indices = index.search(query_embedding, k=2)


In [18]:
for idx in indices[0]:
    print(texts[idx])


AI is widely used in medical diagnosis and healthcare.
Artificial intelligence focuses on building intelligent systems.
